# Backup Tools

## How To Pick

### Key Features

- Automation Capabilities
- Reliability and Robustness
- Incremental Backup Support

### Types Of Tools

#### Open Source Standards

- pg_dump / pg_dumpall: built in logical backups
- pg_basebackup: built in PITR solution for snapshots
- pgBackRest: very commonly used (community standard) open source tool that have almost all capabilities needed, required a bit of expertise (uses postgres low level API)
- Barman: EDBs open source tool for backup and recovery
- Percona: tested pg open source tools tested to work together and bundled, enterprise ready open source solution (partnered with Red Hat, AWS, VMWare and more)

#### Managed (paid) Tools

Most of those are no-code or little code to use
- Cloud native backup tools for managed SQL deployments
- SimpleBackups
- EDB (managed)